# Object Detection with Ultralytics YOLO

This notebook demonstrates object detection using Ultralytics YOLOv8.

**Library:** [Ultralytics](https://ultralytics.com/) - State-of-the-art YOLO implementation

YOLO (You Only Look Once) is a real-time object detection system that can detect multiple objects in a single forward pass.

In [ ]:
import os
import numpy as np
from PIL import Image, ImageDraw
from ultralytics import YOLO
import matplotlib.pyplot as plt

%matplotlib inline

## YOLO Model Overview

Available YOLOv8 model sizes (speed vs accuracy trade-off):

| Model | Size | Speed | Accuracy |
|-------|------|-------|----------|
| yolov8n | Nano | Fastest | Lower |
| yolov8s | Small | Fast | Good |
| yolov8m | Medium | Balanced | Better |
| yolov8l | Large | Slower | High |
| yolov8x | XLarge | Slowest | Highest |

Task-specific models:
- Detection: `yolov8n.pt`
- Segmentation: `yolov8n-seg.pt`
- Classification: `yolov8n-cls.pt`
- Pose Estimation: `yolov8n-pose.pt`

## Create Sample Image

Create a sample scene with drawable objects for demonstration.

In [ ]:
# Create a sample scene image
width, height = 640, 480
sample_image = Image.new("RGB", (width, height), color=(135, 206, 235))  # Sky blue
draw = ImageDraw.Draw(sample_image)

# Draw ground
draw.rectangle([0, int(height * 0.7), width, height], fill=(34, 139, 34))  # Green grass

# Draw a simple "car"
car_x, car_y = 100, 300
draw.rectangle([car_x, car_y, car_x + 150, car_y + 60], fill=(255, 0, 0))  # Body
draw.rectangle([car_x + 20, car_y - 30, car_x + 130, car_y], fill=(255, 0, 0))  # Top
draw.ellipse([car_x + 20, car_y + 40, car_x + 50, car_y + 70], fill=(30, 30, 30))  # Wheel
draw.ellipse([car_x + 100, car_y + 40, car_x + 130, car_y + 70], fill=(30, 30, 30))  # Wheel

# Draw a simple "person"
person_x, person_y = 400, 280
draw.ellipse([person_x - 15, person_y - 50, person_x + 15, person_y - 20], fill=(255, 218, 185))  # Head
draw.rectangle([person_x - 20, person_y - 20, person_x + 20, person_y + 50], fill=(0, 0, 255))  # Body
draw.rectangle([person_x - 10, person_y + 50, person_x + 10, person_y + 100], fill=(0, 0, 139))  # Legs

# Draw sun
draw.ellipse([width - 100, 30, width - 30, 100], fill=(255, 255, 0))

# Draw clouds
for cx, cy in [(150, 60), (180, 50), (210, 60), (180, 70)]:
    draw.ellipse([cx - 25, cy - 20, cx + 25, cy + 20], fill=(255, 255, 255))

print(f"Sample scene created: {sample_image.size}")

plt.figure(figsize=(10, 8))
plt.imshow(sample_image)
plt.title("Sample Scene")
plt.axis("off")
plt.show()

## Load YOLO Model

Load a pretrained YOLOv8 model. The model will be downloaded automatically on first use.

In [ ]:
# Load pretrained YOLOv8 nano model (smallest, fastest)
print("Loading YOLOv8n model...")
model = YOLO("yolov8n.pt")

print(f"Model loaded successfully!")
print(f"Model type: {type(model).__name__}")
print(f"Task: {model.task}")

## Model Information

Explore the model's class names and capabilities.

In [ ]:
# Get class names the model can detect
class_names = model.names
print(f"Number of classes: {len(class_names)}")
print(f"\nSample classes (first 20):")
for i, name in list(class_names.items())[:20]:
    print(f"  {i}: {name}")

## Run Inference

Run object detection on our sample image.

In [ ]:
# Run inference on the sample image
img_array = np.array(sample_image)
results = model(img_array, verbose=False)

# Get the first result
result = results[0]
print(f"Image shape: {result.orig_shape}")
print(f"Inference speed: {result.speed}")

In [ ]:
# Analyze detections
boxes = result.boxes
print(f"\nDetections found: {len(boxes)}")

if len(boxes) > 0:
    print("\nDetected objects:")
    for i, box in enumerate(boxes):
        cls_id = int(box.cls[0])
        cls_name = class_names[cls_id]
        conf = float(box.conf[0])
        xyxy = box.xyxy[0].tolist()
        print(f"  {i+1}. {cls_name}: confidence={conf:.2f}, box={[int(x) for x in xyxy]}")
else:
    print("\nNo objects detected in the simple drawn image.")
    print("(YOLO is trained on real photos, not simple drawings)")

In [ ]:
# Display annotated image
annotated_img = result.plot()  # Returns numpy array with annotations

plt.figure(figsize=(10, 8))
plt.imshow(annotated_img)
plt.title("YOLO Detection Results")
plt.axis("off")
plt.show()

## Using YOLO with NumPy Arrays

YOLO accepts various input formats including file paths, numpy arrays, PIL images, and lists.

In [ ]:
# Different input formats
print("YOLO accepts multiple input formats:")
print("  - File path: model('image.jpg')")
print("  - PIL Image: model(pil_image)")
print("  - NumPy array: model(numpy_array)")
print("  - List of images: model([img1, img2, img3])")
print("  - URL: model('https://example.com/image.jpg')")

In [ ]:
# Run inference on numpy array
print(f"Input array shape: {img_array.shape}")
results_np = model(img_array, verbose=False)
print(f"Detections: {len(results_np[0].boxes)}")

## Batch Processing

Process multiple images in a single batch for efficiency.

In [ ]:
# Create multiple images for batch processing
batch_images = []
for i in range(3):
    img = sample_image.copy()
    d = ImageDraw.Draw(img)
    # Add unique marker to each
    d.rectangle([10, 10, 50, 50], fill=(i * 80, 100, 200))
    batch_images.append(np.array(img))

# Run batch inference
batch_results = model(batch_images, verbose=False)

print(f"Processed {len(batch_results)} images in batch")
for i, res in enumerate(batch_results):
    print(f"  Image {i+1}: {len(res.boxes)} detections")

## Model Export Options

YOLO models can be exported to various formats for deployment.

In [ ]:
print("YOLO models can be exported to various formats:")
print("")
print("  model.export(format='onnx')        # ONNX format")
print("  model.export(format='torchscript') # TorchScript")
print("  model.export(format='tflite')      # TensorFlow Lite")
print("  model.export(format='coreml')      # CoreML (iOS)")
print("  model.export(format='engine')      # TensorRT")
print("")
print("Supported formats:")
print("  PyTorch, TorchScript, ONNX, OpenVINO, TensorRT,")
print("  CoreML, TF SavedModel, TF GraphDef, TF Lite,")
print("  TF Edge TPU, TF.js, PaddlePaddle, ncnn")

## Training Custom Models

How to train YOLO on your own dataset.

In [ ]:
print("To train YOLO on custom data:")
print("")
print("1. Prepare dataset in YOLO format:")
print("   dataset/")
print("   ├── train/")
print("   │   ├── images/")
print("   │   └── labels/")
print("   └── val/")
print("       ├── images/")
print("       └── labels/")
print("")
print("2. Create data.yaml:")
print("   train: path/to/train/images")
print("   val: path/to/val/images")
print("   nc: 2  # number of classes")
print("   names: ['class1', 'class2']")
print("")
print("3. Train:")
print("   model = YOLO('yolov8n.pt')")
print("   model.train(data='data.yaml', epochs=100, imgsz=640)")
print("")
print("4. Validate:")
print("   metrics = model.val()")

## YOLO Tasks

Different YOLO tasks available.

In [ ]:
print("YOLO Task Examples:")
print("")
print("Detection (default):")
print("  model = YOLO('yolov8n.pt')")
print("  results = model('image.jpg')")
print("  boxes = results[0].boxes  # Bounding boxes")
print("")
print("Segmentation:")
print("  model = YOLO('yolov8n-seg.pt')")
print("  results = model('image.jpg')")
print("  masks = results[0].masks  # Instance segmentation masks")
print("")
print("Classification:")
print("  model = YOLO('yolov8n-cls.pt')")
print("  results = model('image.jpg')")
print("  probs = results[0].probs  # Classification probabilities")
print("")
print("Pose Estimation:")
print("  model = YOLO('yolov8n-pose.pt')")
print("  results = model('image.jpg')")
print("  keypoints = results[0].keypoints  # Body keypoints")

---

## Summary

In this notebook, we covered:

1. **YOLO Overview**: Model sizes (nano to xlarge), task-specific models
2. **Loading Models**: Automatic download of pretrained weights
3. **Running Inference**: On images, numpy arrays, batches
4. **Analyzing Results**: Boxes, classes, confidence scores
5. **Export Options**: ONNX, TensorRT, CoreML, TFLite, etc.
6. **Training**: Custom dataset format and training workflow
7. **Tasks**: Detection, Segmentation, Classification, Pose

For more information:
- [Ultralytics Documentation](https://docs.ultralytics.com/)
- [YOLOv8 GitHub](https://github.com/ultralytics/ultralytics)
- [YOLO Training Guide](https://docs.ultralytics.com/modes/train/)